In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,28840
2,Huelva,Confirmados PDIA 14 días,1039
3,Huelva,Tasa PDIA 14 días,"202,46701872673773"
4,Huelva,Confirmados PDIA 7 días,462
5,Huelva,Total Confirmados,29016
6,Huelva,Curados,25224
7,Huelva,Fallecidos,365


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  28840.0
<ipython-input-7-31b732756b19>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  7497.0
<ipython-input-8-e81bfb40e710>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

<ipython-input-9-f09dd2050cc1>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
<ipython-input-9-f09dd2050cc1>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1081 personas en los últimos 7 días 

Un positivo PCR cada 549 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,28840.0,462.0,1039.0,513170.0,90.028645,202.467019,161.0
Huelva-Costa,16800.0,215.0,436.0,289548.0,74.253664,150.579524,96.0
Condado-Campiña,9130.0,215.0,514.0,156231.0,137.616734,329.000006,76.0
Huelva (capital),7497.0,133.0,262.0,143837.0,92.465777,182.150629,57.0
Almonte,1564.0,70.0,146.0,24507.0,285.632676,595.748154,24.0
Sierra de Huelva-Andévalo Central,2568.0,30.0,80.0,67391.0,44.516330,118.710214,18.0
Bollullos Par del Condado,848.0,46.0,85.0,14387.0,319.733092,590.811149,17.0
Isla Cristina,2074.0,16.0,37.0,21393.0,74.790819,172.953770,11.0
Cartaya,1282.0,21.0,45.0,20083.0,104.566051,224.070109,11.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Lucena del Puerto,233.0,6.0,42.0,3261.0,183.992640,1287.948482,0.0
Cumbres de San Bartolomé,11.0,1.0,3.0,376.0,265.957447,797.872340,0.0
Almonte,1564.0,70.0,146.0,24507.0,285.632676,595.748154,24.0
Bollullos Par del Condado,848.0,46.0,85.0,14387.0,319.733092,590.811149,17.0
Beas,201.0,5.0,22.0,4341.0,115.180834,506.795669,3.0
Cañaveral de León,5.0,1.0,2.0,396.0,252.525253,505.050505,0.0
Galaroza,30.0,3.0,6.0,1382.0,217.076700,434.153401,2.0
Bonares,256.0,4.0,20.0,6060.0,66.006601,330.033003,1.0
Condado-Campiña,9130.0,215.0,514.0,156231.0,137.616734,329.000006,76.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Trigueros,268.0,1.0,15.0,7862.0,12.719410,190.791147,1.0,0.066667
Lucena del Puerto,233.0,6.0,42.0,3261.0,183.992640,1287.948482,0.0,0.142857
Cortegana,200.0,1.0,7.0,4602.0,21.729683,152.107779,1.0,0.142857
Bonares,256.0,4.0,20.0,6060.0,66.006601,330.033003,1.0,0.200000
Minas de Riotinto,114.0,1.0,5.0,3812.0,26.232949,131.164743,2.0,0.200000
Beas,201.0,5.0,22.0,4341.0,115.180834,506.795669,3.0,0.227273
San Juan del Puerto,451.0,6.0,26.0,9411.0,63.755180,276.272447,1.0,0.230769
Punta Umbría,717.0,7.0,22.0,15355.0,45.587756,143.275806,4.0,0.318182
Almonaster la Real,66.0,1.0,3.0,1788.0,55.928412,167.785235,0.0,0.333333
